<a href="https://colab.research.google.com/github/joemarshall/websensors/blob/main/assets/python/TempoTap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab workbook presents a workflow for training a machine learning model for a simple pattern based recognition, then outputs it as a tflite file which can be used in the websensor platform or on a raspberry pi.

In [ ]:
# tensorflow is the machine learning library we use
import tensorflow as tf
# numpy is for fast python maths
import numpy as np
# pandas for importing datafiles
import pandas as pd
import io

# make some stuff that is in tensorflow be 
# easier to get at below
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.losses as losses




In [ ]:
# load datafiles - each datafile is a csv file of continuous sensor data. 
# The accompanying ground truth is 1 during a correct unlock sequence, 2 during 
# a failed unlock sequence, 0 otherwise

# recorded data can be from a raspberry pi with two buttons which are held down during 
# the knock sequences, or from the websensor platform

# this stuff makes an upload box appear
from google.colab import files
uploaded = files.upload()

Saving bad_ws.csv to bad_ws (2).csv
Saving good_ws.csv to good_ws (2).csv


In [ ]:
# preprocess - for each knock sequence take the time that the 
# sequence takes and spread the raw data over 512 points
# and normalise to 0-1 to avoid differences in sensitivity between sensors
# or hardness of knock
# We also trim silence off the knock pattern
MIN_SAMPLES=50
column_names=["sound level","ground truth"]

all_knocks=[]

def preprocess_knock(sensor_data,gt_val):
  # normalize the value of sensor data
  sensor_np=np.array(sensor_data)
  # trim silence off the end
  last_sound_pos=np.argmax(np.flip(sensor_np)>128)
  sensor_np=sensor_np[:-last_sound_pos]
  sensor_np=sensor_np-np.min(sensor_np) # min is now zero
  max_sensor=np.max(sensor_np)
  if max_sensor>0:
    sensor_np=sensor_np/max_sensor # max is now one

  # resample it to 512 samples long
  x_out_positions=np.linspace(0,512,512)
  x_original=np.linspace(0,512,len(sensor_np))
  sensor_resampled=np.interp(x_out_positions,x_original,sensor_np)
  # make it have 2 axes - time point, sensor, because this is what tensorflow expects
  sensor_resampled=np.expand_dims(sensor_resampled,-1)
  # we want only zero and one, because we don't use any bits where no 
  # button is pressed
  gt_val=gt_val-1
  return sensor_resampled,gt_val

for c in uploaded.keys():
  print(f"Loading: {c}")
  csv_frame=pd.read_csv(io.BytesIO(uploaded[c]))
  last_gt=0
  press_buffer=None
  for i,row in csv_frame.iterrows():
    gt=row[column_names[1]]
    sensor=row[column_names[0]]
    if gt!=last_gt:
      if gt==0:
        # finished pressing - add this to our list of knocks
        # as long as it is more than 50 samples
        # otherwise assume error
        if len(press_buffer)>MIN_SAMPLES:
          # add this knock to our data
          all_knocks.append(preprocess_knock(press_buffer,last_gt))
      else:
        # starting a press, clear the press buffer
        press_buffer=[]
    if gt!=0:
      press_buffer.append(sensor)
    last_gt=gt

# make arrays for x and y
x_data=np.stack([x for (x,y) in all_knocks])
y_data=np.stack([y for (x,y) in all_knocks])
print(f"Loaded data: {x_data.shape},{y_data.shape}")

Loading: bad_ws.csv
Loading: good_ws.csv
Loaded data: (149, 512, 1),(149,)


In [ ]:
# maybe could do with some data augmentation?
# add silence at the start and end so things shift around a bit
# 

In [ ]:
# shuffle the datasets
p = np.random.permutation(x_data.shape[0])
x_data=x_data[p]
y_data=y_data[p]

# split the datasets into train and test
split_point=int (x_data.shape[0]*.75 )
#split_point=x_data.shape[0]-1
x_train=x_data[0:split_point]
x_test=x_data[split_point:]
y_train=y_data[0:split_point]
y_test=y_data[split_point:]


In [ ]:
# build a model - 4 convolutional layers to identify features, then a fully connected layer to output 
# the unlock or not unlock inference
model=keras.Sequential(layers=[layers.Input(name='x',shape=(512,1)),layers.Conv1D(32,kernel_size=3,padding="same",strides=2,activation="relu"),
                         layers.Conv1D(32,kernel_size=3,padding="same",strides=2,activation="relu"),
                         layers.Conv1D(32,kernel_size=3,padding="same",strides=2,activation="relu"),
                         layers.Conv1D(32,kernel_size=3,padding="same",strides=2,activation="relu"),
                         #layers.Conv1D(64,kernel_size=32,padding="same",strides=32,activation="relu"),
                         layers.Flatten(),
                         layers.Dense(64,activation="relu"),
                         layers.Dense(2,activation="softmax",name='y')])
model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
model.build(input_shape=(None,512,1))
model.summary()
print(model.input,model.output)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 256, 32)           128       
                                                                 
 conv1d_5 (Conv1D)           (None, 128, 32)           3104      
                                                                 
 conv1d_6 (Conv1D)           (None, 64, 32)            3104      
                                                                 
 conv1d_7 (Conv1D)           (None, 32, 32)            3104      
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 64)                65600     
                                                                 
 y (Dense)                   (None, 2)                

In [ ]:
# call train on the model
model.fit(x_train,y_train,batch_size=1,validation_data=(x_test,y_test),epochs=100)


Epoch 1/100
111/111 [==============================] - 1s 6ms/step - loss: 0.6832 - accuracy: 0.5495 - val_loss: 0.7278 - val_accuracy: 0.5789
Epoch 2/100
111/111 [==============================] - 0s 4ms/step - loss: 0.5566 - accuracy: 0.7838 - val_loss: 0.6974 - val_accuracy: 0.6316
Epoch 3/100
111/111 [==============================] - 0s 4ms/step - loss: 0.4516 - accuracy: 0.7928 - val_loss: 0.9115 - val_accuracy: 0.6316
Epoch 4/100
111/111 [==============================] - 1s 5ms/step - loss: 0.3653 - accuracy: 0.8649 - val_loss: 0.7676 - val_accuracy: 0.6053
Epoch 5/100
111/111 [==============================] - 1s 5ms/step - loss: 0.3148 - accuracy: 0.8378 - val_loss: 0.9095 - val_accuracy: 0.6579
Epoch 6/100
111/111 [==============================] - 0s 4ms/step - loss: 0.2436 - accuracy: 0.9009 - val_loss: 0.9294 - val_accuracy: 0.6579
Epoch 7/100
111/111 [==============================] - 0s 4ms/step - loss: 0.2178 - accuracy: 0.9009 - val_loss: 1.0522 - val_accuracy: 0.5789

In [ ]:
#plot confusion matrix so we can see how it predicts for different categories
test_prediction=tf.math.argmax(model.predict(x_test),axis=-1)
train_prediction=tf.math.argmax(model.predict(x_train),axis=-1)
print("Test predictions")
print(tf.math.confusion_matrix(y_test,test_prediction))
print("train predictions")
print(tf.math.confusion_matrix(y_train,train_prediction))

Test predictions
tf.Tensor(
[[17  5]
 [ 6 10]], shape=(2, 2), dtype=int32)
train predictions
tf.Tensor(
[[50  0]
 [ 2 59]], shape=(2, 2), dtype=int32)


In [ ]:
# Save model to a tflite model for inference on raspberry pi (or websensor platform)
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite=converter.convert()

tflite_model_file = open('model.tflite',"wb")
tflite_model_file.write(tflite)

interpreter = tf.lite.Interpreter(model_content=tflite)

signatures = interpreter.get_signature_list()
print(signatures)

from google.colab import files
files.download('model.tflite')


INFO:tensorflow:Assets written to: /tmp/tmprvhrkwc_/assets


INFO:tensorflow:Assets written to: /tmp/tmprvhrkwc_/assets


{'serving_default': {'inputs': ['x'], 'outputs': ['y']}}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>